<a href="https://colab.research.google.com/github/Kimyobu/colab-remote/blob/main/CiscoFantasyLiteColabServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📜 ข้อตกลงการใช้งาน (User Terms) / Nutzungsbedingungen

### 🇹🇭 ภาษาไทย

โดยการกด “ตกลง (Agree)” เพื่อใช้งาน Notebook นี้  
คุณยินยอมให้:

- 📂 Notebook เข้าถึง Google Drive ของคุณ เพื่อใช้เก็บข้อมูลเซิร์ฟเวอร์ Minecraft (เฉพาะโฟลเดอร์ที่ระบุ)  
- 🔑 ใส่ NGROK Auth Token เพื่อให้สามารถเปิดการเชื่อมต่อแบบ Remote TCP ได้  
- 🚫 ห้ามนำระบบนี้ไปใช้ในทางที่ผิด เช่น เซิร์ฟเวอร์ละเมิดลิขสิทธิ์, แจกจ่าย modpack เถื่อน หรือขัดต่อนโยบายของ Mojang, Google หรือ NGROK  
- ⚠️ ข้อมูลที่ใช้งานร่วมกันอาจถูกลบได้โดยไม่แจ้งล่วงหน้า หากมีการใช้งานผิดวัตถุประสงค์  
- 📌 ผู้ใช้ต้องรับผิดชอบต่อข้อมูลและลิขสิทธิ์ของ modpack ที่นำมาใช้งานเอง

หากคุณไม่ยินยอมตามข้อตกลงนี้ กรุณา **อย่าใช้งาน Notebook นี้**

---

### 🇩🇪 Deutsch

Durch das Klicken auf **"Zustimmen (Agree)"** erklärst du dich einverstanden:

- 📂 Dieses Notebook darf auf dein Google Drive zugreifen, um Minecraft-Serverdateien zu speichern (nur im festgelegten Ordner).  
- 🔑 Du musst einen gültigen NGROK Auth Token angeben, um den TCP-Tunnel zu aktivieren.  
- 🚫 Keine Nutzung für illegale Zwecke wie urheberrechtswidrige Server, illegale Modpack-Verbreitung oder Verstöße gegen die Richtlinien von Mojang, Google oder NGROK.  
- ⚠️ Gemeinsame Daten können ohne Vorwarnung gelöscht werden, wenn Missbrauch festgestellt wird.  
- 📌 Du trägst die Verantwortung für die Inhalte und Lizenzen der verwendeten Modpacks.

Wenn du diesen Bedingungen **nicht zustimmst**, verwende dieses Notebook bitte **nicht**.


In [ ]:
#@title <font size='7'><b>Installer. <font size='5'>Run <font color="red"><u>once</u> <font color="white">to install on <font color="yellow">Google Drive
agree_user_term_policy = True # @param {"type":"boolean"}
#@markdown Read <a href="#scrollTo=rxnRRzB2c9-3">User Terms</a>
if agree_user_term_policy is False:
  raise UserWarning("Please read and argee 'User Terms'")
force_reinstall = False # @param {"type":"boolean"}
# @markdown To force reinstall enable this option. <font color=red><b>(Server Data Will Be WIPE)
from google.colab import drive, output
import os

if (os.environ.get("_env") != "True"):
  confirm_reinstall = False
  _pyngrok = False
  _install = False

if os.environ.get("_env") is None:
  os.environ["_env"] = "True"
if not os.path.exists("/content/drive/MyDrive"):
    drive.mount("/content/drive", force_remount=True)
PROJECT = "/content/drive/MyDrive/CiscoFantasyLite_ColabServer"
# PROJECT = "/content/CiscoFantasyLite_ColabServer"
if not os.path.exists(PROJECT):
    os.mkdir(PROJECT)
    _install = True
if force_reinstall is True:
  if (confirm_reinstall is True):
    print("Wiping All Data...")
    !rm -r $PROJECT
    os.mkdir(PROJECT)
    _install = True
    confirm_reinstall = False
  else:
    confirm_reinstall = True
    raise Exception("Reconfirmation Again.")

if (_pyngrok is not True):
  %pip install -q pyngrok
  _pyngrok = True

if (_install is True):
  print("Installing...")
  %cd $PROJECT
  !wget -c "https://www.curseforge.com/api/v1/mods/873717/files/5894650/download" -O "server_pack.zip"
  !unzip server_pack.zip
  !rm server_pack.zip
  !wget -c https://download.oracle.com/java/17/archive/jdk-17.0.12_linux-x64_bin.tar.gz -O jdk.tar.gz
  !tar -xvzf jdk.tar.gz
  !rm jdk.tar.gz
  _install = False
output.clear()
print("All done.")

In [ ]:
#@title <font size=6><b>Start Server.
agree_user_term_policy = False # @param {"type":"boolean"}
#@markdown Read <a href="#scrollTo=rxnRRzB2c9-3">User Terms</a>
if agree_user_term_policy is False:
  raise UserWarning("Please read and argee 'User Terms'")
from google.colab import drive, output
import os
if not os.path.exists("/content/drive/MyDrive"):
    drive.mount("/content/drive", force_remount=True)
PROJECT = "/content/drive/MyDrive/CiscoFantasyLite_ColabServer"
# PROJECT = "/content/CiscoFantasyLite_ColabServer"
if os.path.exists(PROJECT) is not True:
  raise UserWarning("Project Directory Not Found. Please run Installer First.")
%cd $PROJECT
!sed -i 's/JAVA_ARGS="-Xmx[0-9]\+G -Xms[0-9]\+G"/JAVA_ARGS="-Xmx10G -Xms10G"/' variables.txt
!sed -i 's|^JAVA=".*"|JAVA="./jdk-17.0.12/bin/java"|' variables.txt
!chmod +x start.sh
from pyngrok import ngrok, conf
import time
ngrok_auth_token = "" # @param {"type":"string","placeholder":"Your auth token key..."}
#@markdown Get it from https://dashboard.ngrok.com/get-started/your-authtoken
region = "eu" # @param ["eu","ap"]
!ngrok config add-authtoken $ngrok_auth_token
conf.get_default().region = region
tcp_tunnel = ngrok.connect(25565, "tcp")
print("Here is Tunnel Url:", tcp_tunnel)
print("Server Will Start in 5s...")
time.sleep(5)
!./start.sh